# Trial For SAFE Code

## Intro

This notebook is a playground for testing various implementations of the SAFE protocol in an *object-oriented* fashion.

We begin with what our 'main' might look like. 

Let's start simple : 3 users 

In [2]:
import random as r
from itertools import repeat
import numpy as np
from MoodAppUser import MoodAppUser

# from `safe-applied-to-use-case.ipynb`

NO_OF_DAYS_TRACKED = 7
NO_OF_USERS = 3

KEYWORDS = [0, 1, 2, 3, 4, 5, 6] # corresponding to our 7-emotion taxonomy

random_document_sets = [r.choices(KEYWORDS, k = NO_OF_DAYS_TRACKED) for _ in repeat(None, NO_OF_USERS)]

users = [MoodAppUser(i, document_set) for i, document_set in enumerate(random_document_sets)]

print("Here are some users and their raw feature vectors:\n")
for i in range(3):
    print(f"User {users[i].id} and their raw feature vectors:\n{users[i].feature_vector} of length {len(users[i].feature_vector)}\n")

Here are some users and their raw feature vectors:

User 0 and their raw feature vectors:
[0.4286 0.1429 0.1429 0.1429 0.1429 0.     0.    ] of length 7

User 1 and their raw feature vectors:
[0.1429 0.1429 0.2857 0.1429 0.1429 0.1429 0.    ] of length 7

User 2 and their raw feature vectors:
[0.2857 0.4286 0.1429 0.1429 0.     0.     0.    ] of length 7



#### Uniform Priors Generation

We generate some uniform priors for the bayesian calculation

In [3]:
no_of_keywords = len(KEYWORDS)
priors_for_keywords = [round(1 / no_of_keywords, 4) for _ in repeat(None, no_of_keywords)]

#### N Shares Generation

Users first want to generate $N-1$ shares to send to other users. 

They will also generate their $N^{th}$ share (to keep) using these.

First, we set the D value for the round, and then users will draw shares from these.

For sake of simplicity, we will defer randomized generation of D to later.

In [4]:
MoodAppUser.set_D_value()

for i in range(NO_OF_USERS):
    users[i].generate_shares_to_send(NO_OF_USERS, no_of_keywords)
    users[i].calculate_Nth_share()

# SANITY CHECK
assert np.all(np.subtract(users[0].feature_vector, np.sum(list(users[0].shares_to_send.values()), axis = 0))  == users[0].Nth_share) 

#### Share Distribution

Now we wish to distribute the shares among the users.

User 0 will send the shares they generated for User 1 and User 2 to each respectively, and so on.

In [5]:
for user in range(NO_OF_USERS):
    other_user_ids = [id for id in range(NO_OF_USERS) if id != user]
    for other_user in other_user_ids:
        share_to_send = users[user].get_share_for_user(other_user)
        users[other_user].receive_share(share_to_send)
    # NOTE - as noted below, we can generate the obfuscated vector here if we want to be more efficient.

# SANITY CHECK
assert np.all(users[0].shares_to_send[1] == users[1].shares_received[0])

### Calculate Obfuscated Feature Vector

Having distributed the shares, each user can now calculate their obfuscated feature vector.

They do this by adding the sum of received shares to their Nth share.

In [6]:
for user in range(NO_OF_USERS):
    users[user].generate_obfuscated_feature_v()

# NOTE - share distribution and obfuscated vector generation can be done in the same loop

#### Secure Aggregation

Let's now see if aggregating the obfuscated feature vectors results in the same result as aggregating the raw feature vectors. If so, our protocol has been implemented correctly.

In [9]:
# Computing Target Function: Aggregate Sum

target_result = np.sum(list([user.feature_vector for user in users]), axis = 0)
masked_vector_result = np.sum(list([user.obfuscated_feature_v for user in users]), axis = 0)

# if we need to be extra precise :
target_result = np.round(target_result, 4) 
masked_vector_result = np.round(masked_vector_result, 4)
assert np.array_equal(target_result, masked_vector_result)

print(f"Sum of raw vectors: \n\n {target_result}")
print(f"Sum of masked vectors: \n\n {masked_vector_result}\n")

Sum of raw vectors: 

 [0.8572 0.7144 0.5715 0.4287 0.2858 0.1429 0.    ]
Sum of masked vectors: 

 [0.8572 0.7144 0.5715 0.4287 0.2858 0.1429 0.    ]

